In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression , ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import Ridge



In [8]:
#importing the data 
processed_train_df = pd.read_csv("C:/Users/Mihir S Kagalkar/OneDrive - iiit-b/SEM-5/ML/PROJECT/jupyter_files/Processed_train_data.csv")
processed_test_df = pd.read_csv("C:/Users/Mihir S Kagalkar/OneDrive - iiit-b/SEM-5/ML/PROJECT/jupyter_files/Processed_test_data.csv")
train_df = train_df = pd.read_csv("C:/Users/Mihir S Kagalkar/OneDrive - iiit-b/SEM-5/ML/PROJECT/Hotel-Property-Value-Dataset/train.csv")

In [22]:
#initial tree+linear ridge model
# --- PARAMETERS ---
target_col = 'HotelValue'
# ---------------------------------------
# Separate features and target
# ---------------------------------------
X = processed_train_df.copy()
y = train_df[target_col].values.reshape(-1, 1)

# Scale target (stable regression at leaves)
y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y).flatten()

# Scale ALL features (numeric + categorical + PCA output)
X_scaler = MinMaxScaler()
X_scaled = X_scaler.fit_transform(X)

# ---------------------------------------
# K-Fold Decision Tree + Linear Regression at leaves
# ---------------------------------------
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = []
# Hyperparameters (you can tune later)
max_depth = 8
min_samples_leaf = 100
top_k_features = 10  # number of features allowed in leaf regression
ridge_alpha = 1    # regularization strength

for train_index, val_index in kf.split(X_scaled):
    # Split fold
    X_train_fold = X_scaled[train_index]
    y_train_fold = y_scaled[train_index]
    X_val_fold = X_scaled[val_index]
    y_val_fold = y_scaled[val_index]

    # Fit decision tree
    tree = DecisionTreeRegressor(
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    tree.fit(X_train_fold, y_train_fold)

    # Get top-k important features
    feat_idx_sorted = np.argsort(tree.feature_importances_)[::-1]
    top_k_idx = feat_idx_sorted[:top_k_features]

    # Train a regression model for each leaf
    leaf_train_ids = tree.apply(X_train_fold)
    leaf_models = {}

    for leaf in np.unique(leaf_train_ids):
        leaf_idx = np.where(leaf_train_ids == leaf)[0]
        X_leaf = X_train_fold[leaf_idx][:, top_k_idx]
        y_leaf = y_train_fold[leaf_idx]

        # If leaf has too few samples — fallback to mean prediction
        if X_leaf.shape[0] <= X_leaf.shape[1] + 2:
            leaf_models[leaf] = ("mean", float(y_leaf.mean()))
        else:
            lr = Ridge(alpha=ridge_alpha)
            lr.fit(X_leaf, y_leaf)
            leaf_models[leaf] = ("ridge", lr)

    # ---- Prediction ----
    leaf_val_ids = tree.apply(X_val_fold)
    y_pred_scaled = np.zeros_like(y_val_fold)

    for i, leaf_id in enumerate(leaf_val_ids):
        model_type, model = leaf_models.get(
            leaf_id, ("mean", np.mean(y_train_fold))
        )

        if model_type == "mean":
            y_pred_scaled[i] = model
        else:
            row = X_val_fold[i:i+1][:, top_k_idx]
            y_pred_scaled[i] = model.predict(row)[0]

    # Inverse-scale predicted y
    y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1))
    y_true = y_scaler.inverse_transform(y_val_fold.reshape(-1, 1))

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    rmse_scores.append(rmse)

# Evaluation
print("RMSE scores:", rmse_scores)
print("Mean RMSE:", np.mean(rmse_scores))
print("Std RMSE:", np.std(rmse_scores))

RMSE scores: [np.float64(86975.41183737198), np.float64(74199.07805518305), np.float64(71562.34749794248), np.float64(82811.00635794386), np.float64(89500.46124280611)]
Mean RMSE: 81009.6609982495
Std RMSE: 7022.314281913285
